In [1]:
import os
import sys
import torch
from tqdm import tqdm
sys.path.insert(0, os.path.abspath('..'))

In [2]:
from includes.models_pretrained import get_unet_model

In [3]:
from includes.dataloader import prepare_loaders

In [4]:
train_data, val_data = prepare_loaders(batchSize = 4, device = "cuda", numWorkers = 2, shuffle = False, csvFile = "../data/train.csv", basePath = '../data/', shape = (200, 200))

In [5]:
from includes.utils.loss import IoULoss

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [7]:
model = get_unet_model("vanilla", 1, 3, None, device)

In [8]:
# HYPERPARAMETERS
EPOCHS = 40
BATCH_SIZE = 32
LR = 0.0001
WORKERS = 2
history = []

In [12]:
def train_fn(model, dataloader, optimizer, loss_fn):

    loop = tqdm(dataloader)

    for index, (data, target) in enumerate(loop):
        data = data.to(device=device)
        target = target.to(device=device)
        output = model(data)
        loss = loss_fn(output, target)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())
        #loop.set_postfix(j_coer=jaccard(output.cpu().detach().numpy(), target))
        
    history.append(loss.item())


def train_mask():

    loss_fn = IoULoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    
    #check_accuracy(val_loader, model, device=DEVICE)

    for epoch in range(EPOCHS):
        train_fn(model, train_data, optimizer, loss_fn)

In [13]:
train_mask()

  3%|▎         | 112/3318 [02:33<1:13:23,  1.37s/it, loss=1.4e-45]


KeyboardInterrupt: 